In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import swifter
from tqdm import tqdm

In [2]:
def to_lastname(name):
    names_split = name.split(' ')
    if len(names_split) == 1:
        return name
    return ' '.join(names_split[1:])

In [3]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].swifter.apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].swifter.apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [4]:
def merge_csv(country):
    fifa_players = pd.read_csv('fifa21-players-cleaned.csv')
    country_players = pd.read_csv(f'teams/{country}.csv')

    # pull out last names
    fifa_players.Name = fifa_players.Name.apply(to_lastname, 1)
    country_players.Player = country_players.Player.apply(to_lastname, 1)

    # create string_index to compare on
    country_players['string_index'] = country_players.Player + ' ' + country_players.Club
    fifa_players['string_index'] = fifa_players.Name + ' ' + fifa_players.Club

    # merge country players with fifa stats
    new_merge = fuzzy_merge(country_players, fifa_players, 'string_index', 'string_index')
    new_merge.merge(fifa_players, how='left', left_on='matches', right_on='string_index').to_csv(f'team-stats/{country}.csv')

In [5]:
import os

countries = [f[:-4] for f in os.listdir('teams')[1:]]
for country in tqdm(countries[:2]):
    merge_csv(country)

  0%|                                         | 0/2 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/28 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/28 [00:00<?, ?it/s]

 50%|████████████████▌                | 1/2 [00:16<00:16, 16.73s/it]

Pandas Apply:   0%|          | 0/23 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23 [00:00<?, ?it/s]

100%|█████████████████████████████████| 2/2 [00:30<00:00, 15.39s/it]
